In [3]:
import re
import pandas as pd
import numpy as np
import math
import itertools

### Day 1

In [ ]:
f = open('../data/day_1_input.txt', 'r')

sum = 0

for line in f:
    nums = re.findall(r'\d', line)
    sum = sum + int(str(nums[0])+str(nums[-1]))

sum

In [ ]:
f = open('../data/day_1_input.txt', 'r')

sum = 0

digit_dict = {
    'one':1,
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'seven':7,
    'eight':8,
    'nine':9
    }

for line in f:
    nums = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', line)
    for value in range(len(nums)):
        if nums[value] in digit_dict.keys():
            nums[value] = digit_dict[nums[value]]
    sum = sum + int(str(nums[0])+str(nums[-1]))

sum

### Day 2

In [ ]:
df = pd.DataFrame(columns = ['game_id', 'draw_id', 'red', 'blue', 'green'])

In [ ]:
f = open('../data/day_2_input.txt', 'r')

df = pd.DataFrame(columns = ['game_id', 'draw_id', 'red', 'blue', 'green'])

for line in f: 
    
    game_dict = {}

    colon = re.search(':', line).start()
    game_id = int(re.findall(r'\d+',line[:colon])[0])
    game_dict['game_id'] = game_id

    match = re.finditer(';', line)
    draws = [colon] + [m.start() for m in match] + [len(line)]

    for d in range(len(draws)-1):
        draw_str = line[draws[d]+1:draws[d+1]] 
        draw_id = d
        game_dict['draw_id'] = d

        colors = re.findall(r'blue|green|red', draw_str)
        scores = re.findall(r'\d+', draw_str)
        scores = [int(s) for s in scores]

        draw_dict = dict(zip(colors, scores))

        record_dict = game_dict.copy()
        record_dict.update(draw_dict)
        record_dict = {k:[record_dict[k]] for k in record_dict.keys()}
        
        df = pd.concat([df, pd.DataFrame.from_records(record_dict)])

final_df = (
    df
    .fillna(0)
    .assign(is_possible = lambda x: np.where((x['red']<=12) & (x['green']<=13) & (x['blue'] <= 14), 1, 0))
    .groupby('game_id')
    .agg({'is_possible':'min'})
    .reset_index()
    .loc[lambda x: x['is_possible']==1, :]
)

final_df['game_id'].sum()

In [ ]:
final_df = (
    df
    .fillna(0)
    .groupby('game_id')
    .max()
    .reset_index()
    .assign(power = lambda x: x['red']*x['blue']*x['green'])
)

final_df['power'].sum()

### Day 3

In [ ]:
f = open('../data/day_3_input.txt', 'r')

symbol_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'[^A-Za-z0-9.\n]', line)
    symbol_dict[index] = [m.start() for m in match_symbol]
    index = index + 1


In [ ]:
f = open('../data/day_3_input.txt', 'r')

gear_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\*', line)
    gear_dict[index] = [m.start() for m in match_symbol]
    index = index + 1

gear_dict

In [ ]:
f = open('../data/day_3_input.txt', 'r')

index = 0
part_sum = 0

for line in f:
    match_part = re.finditer(r'\d+', line)
    parts = [(m.start(), m.end()) for m in match_part]
    for p in parts:
        is_valid = 0
        value = int(line[p[0]:p[1]])
        if p[0] - 1 in symbol_dict[index]:
            is_valid = 1
        if p[1] in symbol_dict[index]:
            is_valid = 1
        if index < 139:
            for x in symbol_dict[index + 1]:
                if x >= p[0] - 1 and x <= p[1]:
                    is_valid = 1
        if index > 1:
            for x in symbol_dict[index - 1]:
                if x >= p[0] - 1 and x <= p[1]:
                    is_valid = 1
        if is_valid == 1:
            part_sum = part_sum + value
    index = index + 1

part_sum

In [ ]:
f = open('../data/day_3_input.txt', 'r')

part_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\d+', line)
    part_dict[index] = [(m.start(), m.end(), int(line[m.start():m.end()])) for m in match_symbol]
    index = index + 1

In [ ]:
f = open('../data/day_3_input.txt', 'r')

gear_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\*', line)
    gear_dict[index] = [m.start() for m in match_symbol]
    index = index + 1

In [ ]:
gear_sum = 0

for line_index in range(139):
    for i in range(len(gear_dict[line_index])):
        matches = 0
        match_prod = 1
        # match_left
        end_index = [m[1] for m in part_dict[line_index]]
        if gear_dict[line_index][i] in end_index:
            part_value = part_dict[line_index][end_index.index(gear_dict[line_index][i])][2]
            matches = matches + 1
            match_prod = match_prod*part_value
        # match_right
        start_index = [m[0] for m in part_dict[line_index]]
        if gear_dict[line_index][i] + 1 in start_index:
            part_value = part_dict[line_index][start_index.index(gear_dict[line_index][i] + 1)][2]
            matches = matches + 1
            match_prod = match_prod*part_value
        # matches_above
        for m in part_dict[line_index - 1]:
            if not (m[0] >= gear_dict[line_index][i] + 2 or m[1] <= gear_dict[line_index][i] - 1):
                matches = matches + 1
                match_prod = match_prod*m[2]

        # matches_below
        for m in part_dict[line_index + 1]:
            if not (m[0] >= gear_dict[line_index][i] + 2 or m[1] <= gear_dict[line_index][i] - 1):
                matches = matches + 1
                match_prod = match_prod*m[2]

        if matches == 2:
            gear_sum = gear_sum + match_prod

gear_sum


### Day 4

In [ ]:
f = open('../data/day_4_input.txt', 'r')

total_score = 0

for line in f:

    winners = [int(x) for x in re.findall(r'\d+', line[re.search(":", line).end():re.search("\|", line).start()])]
    candidates = [int(x) for x in re.findall(r'\d+', line[re.search("\|", line).end():])]

    matches = 0
    for x in candidates:
        if x in winners:
            matches = matches + 1

    if matches == 0:
        points = 0
    else:
        points = 2**(matches - 1)

    total_score = total_score + points

total_score

In [ ]:
f = open('../data/day_4_input.txt', 'r')

total_score = 0
card_dict = {}

for line in f:

    card_id = int(re.findall(r'\d+', line[:re.search(":", line).start()])[0])
    winners = [int(x) for x in re.findall(r'\d+', line[re.search(":", line).end():re.search("\|", line).start()])]
    candidates = [int(x) for x in re.findall(r'\d+', line[re.search("\|", line).end():])]

    matches = 0
    for x in candidates:
        if x in winners:
            matches = matches + 1

    new_cards = [x+1 for x in range(card_id, card_id + matches)]
    card_dict[card_id] = new_cards

card_list = {x + 1:1 for x in range(206)}

for x in card_list.keys():
    for num in card_list.keys():
        if num in card_dict[x]:
            card_list[num] = card_list[num] + card_list[x]

sum(card_list.values())

### Day 5

In [ ]:
f = open('../data/day_5_input.txt', 'r').read()

def data_map(input, map_df):
    for i in range(map_df.shape[0]):
        if input in range(map_df[i][1], map_df[i][1]+map_df[i][2]):
            return int(map_df[i][0] + input - map_df[i][1])
    return int(input)

cats = [
    'seeds', 
    'seed-to-soil map',
    'soil-to-fertilizer map', 
    'fertilizer-to-water map', 
    'water-to-light map', 
    'light-to-temperature map', 
    'temperature-to-humidity map',
    'humidity-to-location map'
    ]

data_dict = {}

for i in range(8):
    if i == 0:
        data_dict[cats[i]] =  [int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():re.search(cats[i+1], f).start()])]
    else:
        if i < 7:
            data_dict[cats[i]] = np.array([int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():re.search(cats[i+1], f).start()])]).reshape((-1, 3))
        else:
            data_dict[cats[i]] = np.array([int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():])]).reshape((-1, 3))


In [ ]:
min_output = 999999999999999999999999

for i in range(len(data_dict['seeds'])):

    input = data_dict['seeds'][i]

    for x in [x for x in data_dict.keys() if x != 'seeds']:
        output = data_map(input, data_dict[x])
        input = output

    if output < min_output:
        min_output = output

print(min_output)


In [ ]:
seed_list = []
for i in range(int(len(data_dict['seeds'])/2)):
    seed_list.append(range(data_dict['seeds'][2*i], data_dict['seeds'][2*i] + data_dict['seeds'][2*i + 1]))

In [ ]:
# Returning a minimum with a huge step size over all possible seeds. 
# In this case, the square root of the length of each range
min_output = 999999999999999999999999
min_seed = 999999999999999999999999

for i in range(len(seed_list)):
    for j in range(0, len(seed_list[i]), math.floor(math.sqrt(len(seed_list[i])))):
        input = seed_list[i][j]

        for x in [x for x in data_dict.keys() if x != 'seeds']:
            output = data_map(input, data_dict[x])
            input = output

        if output < min_output:
            min_i = i
            min_j = j
            min_seed = seed_list[i][j]
            min_output = output

print('i: ', min_i)
print('j: ', min_j)
print('seed: ', min_seed)
print('output: ', min_output)


In [ ]:
# The step size used for the range that contains this approximate minimum
math.floor(math.sqrt(len(seed_list[5])))

In [ ]:
min_output = 999999999999999999999999
min_seed = 999999999999999999999999

# Fix the list that has the approximate minimum
i = 5

# Each the step size window around the approximate minimum
for j in range(121537743 - 11441, 121537743 + 11441):
    input = seed_list[i][j]

    for x in [x for x in data_dict.keys() if x != 'seeds']:
        output = data_map(input, data_dict[x])
        input = output

    if output < min_output:
        min_i = i
        min_j = j
        min_seed = seed_list[i][j]
        min_output = output

print('i: ', min_i)
print('j: ', min_j)
print('seed: ', min_seed)
print('output: ', min_output)

### Day 6

In [ ]:
f = open('../data/day_6_input.txt', 'r')

data_dict = {}

for line in f:
    print(line)
    data = line.lower().replace(':', '').split()
    data_dict[data[0]] = [int(x) for x in data[1:]]

data_dict

In [ ]:
counter_prod = 1

for race in range(len(data_dict['time'])):
    counter = 0
    for choice in range(data_dict['time'][race]+1):
        if choice*(data_dict['time'][race]-choice) > data_dict['distance'][race]:
            counter = counter + 1
            
    print(counter)
    counter_prod = counter_prod*counter

print(counter_prod)

In [ ]:
ms = ''
for i in range(len(data_dict['time'])):
    ms = ms+str(data_dict['time'][i])

ms = int(ms)

dist = ''
for i in range(len(data_dict['distance'])):
    dist = dist + str(data_dict['distance'][i])

dist = int(dist)

root_1 = (ms + math.sqrt(ms**2 - 4*dist))/2
root_2 = (ms - math.sqrt(ms**2 - 4*dist))/2

len(range(math.ceil(root_2), math.ceil(root_1)))


### Day 7

In [ ]:
hand_value_map = {
    (5, None):(15**3)*6,
    (4, 1):(15**3)*5,
    (3, 2):(15**3)*4,
    (3, 1):(15**3)*3,
    (2, 2):(15**3)*2,
    (2, 1):(15**3)*1,
    (1, 1):(15**3)*0
}

card_map = {
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    'T':11,
    'J':12,
    'Q':13,
    'K':14,
    'A':15
}

card_map_jokers = {
    'J':1,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    'T':11,
    'Q':12,
    'K':13,
    'A':14
}

def hand_value(hand:str, joker = False):
    if joker == True:
        hand = [card_map_jokers[x] for x in list(hand)]
    else:
        hand = [card_map[x] for x in list(hand)]

    hand_grouped = {x:hand.count(x) for x in hand}

    if joker == True:
        wilds = hand_grouped.pop(1, 0)
        
    values_sorted = sorted(hand_grouped.values(), reverse=True)
    if len(values_sorted) > 0:
        largest_group = values_sorted[0]
    else:
        largest_group = 0
    
    if joker == True:
        largest_group = largest_group + wilds

    if largest_group == 5:
        second_largest_group = None
    else:
        second_largest_group = values_sorted[1]

    value = hand_value_map[(largest_group, second_largest_group)]

    tiebreaker = 0
    for i in range(5):
        tiebreaker = tiebreaker + (hand[i]-1)/15**(i-2)

    return value + tiebreaker

In [ ]:
f = open('../data/day_7_input.txt', 'r')

data_dict = {}

i = 0
for line in f:
    data = line.split()
    data_dict.update({i:{}})
    data_dict[i]['hand'] = data[0]
    data_dict[i]['bid'] = int(data[1])
    i = i + 1

df = pd.DataFrame.from_dict(data_dict, orient='index')

part_1 = (df
    .assign(value = df['hand'].apply(hand_value))
    .assign(rank = lambda x: x['value'].rank())
)

sum(part_1['rank']*part_1['bid'])

In [ ]:
part_2 = (df
    .assign(value = df['hand'].apply(hand_value, joker=True))
    .assign(rank = lambda x: x['value'].rank())
)

sum(part_2['rank']*part_2['bid'])

### Day 8

In [ ]:
f = open('../data/day_8_input.txt', 'r')

index = 0
nodes = {}
for line in f:
    if index == 0:
        instructions = line.strip()
    if index >= 2: 
        line = line.strip()
        idx = line.find('=')
        nodes[line[:idx-1]] = line[idx+2:].replace('(', '').replace(')', '').replace(' ', '').split(',')
    index = index + 1

In [ ]:
current_node = 'AAA'
step = 0
while current_node != 'ZZZ' and step < 100000: 
    direction = instructions[step % len(instructions)]
    if direction == 'L':
        current_node = nodes[current_node][0]
    else:
        current_node = nodes[current_node][1]
    step = step + 1

step

In [ ]:
for node in [x for x in nodes.keys() if x[2] == 'A']:
    current_node = node
    step = 0
    while step < 100000: 
        direction = instructions[step % len(instructions)]
        if direction == 'L':
            current_node = nodes[current_node][0]
        else:
            current_node = nodes[current_node][1]
        step = step + 1

        if current_node[2] == 'Z':
            print(step, step % len(instructions), node, current_node)

+ DNA - 20569
+ HNA - 18727
+ AAA - 14429
+ LMA - 13201 
+ VGA - 18113
+ LLA - 22411

In [ ]:
cycles = [20569, 18727, 14429, 13201, 18113, 22411]

In [ ]:
ans = math.lcm(20569, 18727)
for i in range(2,6):
    ans = math.lcm(ans, cycles[i])

ans

### Day 9

In [ ]:
f = open('../data/day_9_input.txt', 'r')

sequences = []
for line in f:
    sequences.append([int(x) for x in line.strip().split()])

In [ ]:
f = open('../data/day_9_input.txt', 'r')

sequences = []
for line in f:
    sequences.append([int(x) for x in line.strip().split()])

ans = 0

for pattern in sequences:
    last_entries = [pattern[len(pattern)-1]]
    diffs = np.array(pattern[1:]) - np.array(pattern[:-1]) 

    while ~np.all(diffs == 0):
        last_entries.append(diffs[len(diffs)-1])
        diffs = np.array(diffs[1:]) - np.array(diffs[:-1])


    ans = ans + sum(last_entries)

ans

In [ ]:
f = open('../data/day_9_input.txt', 'r')

sequences = []
for line in f:
    sequences.append([int(x) for x in line.strip().split()])

ans = 0

for pattern in sequences:
    first_entries = [pattern[0]]
    diffs = np.array(pattern[1:]) - np.array(pattern[:-1]) 
    
    idx = 1
    while ~np.all(diffs == 0):
        first_entries.append(((-1)**idx)*diffs[0])
        diffs = np.array(diffs[1:]) - np.array(diffs[:-1])
        idx = idx + 1

    ans = ans + sum(first_entries)

ans

### Day 10

In [ ]:
f = open('../data/day_10_input.txt', 'r')

grid = []

for line in f:
    grid.append(list(line.strip()))

steps_away = np.zeros(np.array(grid).shape)

In [ ]:
def connected_coords(x,y):
    if grid[y][x] == '|':
        return [[x, y+1], [x, y-1]]
    if grid[y][x] == '-':
        return [[x-1, y], [x+1, y]]
    if grid[y][x] == 'L':
        return [[x+1, y], [x, y-1]]
    if grid[y][x] == 'J':
        return [[x-1, y], [x, y-1]]
    if grid[y][x] == '7':
        return [[x-1, y], [x, y+1]]
    if grid[y][x] == 'F':
        return [[x+1, y], [x, y+1]] 
    else:
        return None

In [ ]:
current_coord = [88, 127]
prior_coord = [88, 128]
n_steps = 1

while grid[current_coord[1]][current_coord[0]] != 'S':
    steps_away[current_coord[1]][current_coord[0]] = n_steps
    for option in connected_coords(current_coord[0], current_coord[1]):
        if option != prior_coord:
            prior_coord = current_coord
            current_coord = option
            n_steps = n_steps + 1
            break


In [ ]:
current_coord = [88, 129]
prior_coord = [88, 128]
n_steps = 1

while grid[current_coord[1]][current_coord[0]] != 'S':
    if steps_away[current_coord[1]][current_coord[0]] != 0:
        if steps_away[current_coord[1]][current_coord[0]] > n_steps:
            steps_away[current_coord[1]][current_coord[0]] = n_steps
    for option in connected_coords(current_coord[0], current_coord[1]):
        if option != prior_coord:
            prior_coord = current_coord
            current_coord = option
            n_steps = n_steps + 1
            break


In [ ]:
steps_away.max()

### Day 11

In [ ]:
f = open('../data/day_11_input.txt', 'r')

grid = []

for line in f:
    grid.append(list(line.strip()))

grid = np.array(grid)

added_rows = 0
for row in range(grid.shape[0]):
    idx = row + added_rows
    if np.all(grid[idx] == '.'):
        grid = np.concatenate((grid[:idx], grid[idx].reshape((1, grid.shape[1])), grid[idx:]), axis = 0)
        added_rows = added_rows + 1

added_cols = 0
for col in range(grid.shape[1]):
    idx = col + added_cols
    if np.all(grid[:, idx] == '.'):
        grid = np.concatenate((grid[:, :idx], grid[:, idx].reshape((grid.shape[0],1)), grid[:, idx:]), axis = 1)
        added_cols = added_cols + 1

galaxies = []
for x in range(grid.shape[1]):
    for y in range(grid.shape[0]):
        if grid[y][x] == '#':
            galaxies.append((x, y))

ans = 0
for pair in itertools.combinations(galaxies, 2):
    ans = ans + abs(pair[0][0] - pair[1][0]) + abs(pair[0][1] - pair[1][1])

ans

In [ ]:
f = open('../data/day_11_input.txt', 'r')

grid = []

for line in f:
    grid.append(list(line.strip()))

grid = np.array(grid)

empty_rows = []
for row in range(grid.shape[0]):
    if np.all(grid[row] == '.'):
        empty_rows.append(row)

empty_cols = []
for col in range(grid.shape[1]):
    if np.all(grid[:, col] == '.'):
        empty_cols.append(col)


galaxies = []
for x in range(grid.shape[1]):
    for y in range(grid.shape[0]):
        if grid[y][x] == '#':
            galaxies.append((x, y))

ans = 0
expansion_factor = 999999
for pair in itertools.combinations(galaxies, 2):
    normal_rows = abs(pair[0][0] - pair[1][0])

    if pair[0][0] < pair[1][0]:
        extra_cols = expansion_factor*len([x for x in empty_cols if x in range(pair[0][0], pair[1][0])])
    else:
        extra_cols = expansion_factor*len([x for x in empty_cols if x in range(pair[1][0], pair[0][0])])

    normal_cols = abs(pair[0][1] - pair[1][1])

    if pair[0][1] < pair[1][1]:
        extra_rows = expansion_factor*len([x for x in empty_rows if x in range(pair[0][1], pair[1][1])])
    else:
        extra_rows = expansion_factor*len([x for x in empty_rows if x in range(pair[1][1], pair[0][1])])

    ans = ans + normal_rows + extra_rows + normal_cols + extra_cols

ans


#### Day 12

In [17]:
ques_options = ['.', '#']
chunk_mem = {}

f = open('../data/day_12_input.txt', 'r')

count = 0
for line in f:
    springs = line.strip().split(' ')[0]
    solution = [int(x) for x in line.strip().split(' ')[1].split(',')]

    min_runs = [x for x in springs.split('.') if x != '']
    chunk_options = []

    for run in range(len(min_runs)):
        chunk_options.append([])

        if min_runs[run] in chunk_mem.keys():
            chunk_options[run] = chunk_mem[min_runs[run]]
        else:
            choices = list(itertools.product([0, 1], repeat=min_runs[run].count('?')))
            replace_index = [i for i, ltr in enumerate(min_runs[run]) if ltr == '?']
            string_to_eval = min_runs[run]

        for choice in choices:
            for i in range(len(choice)):
                string_to_eval = string_to_eval[:replace_index[i]] + ques_options[choice[i]] + string_to_eval[replace_index[i]+1:]
            chunk_options[run].append([len(x) for x in string_to_eval.split('.') if x != ''])

    combs = [range(len(chunk_options[x])) for x in range(len(chunk_options))]
    combs = list(itertools.product(*combs))

    for x in combs:
        ans = []
        for i in range(len(chunk_options)):
            ans = ans + chunk_options[i][x[i]]
        if ans == solution:
            count = count + 1

count

7350

In [36]:
chunk_mem = {}
ques_options = ['.', '#']
count = 0

line = '?###???????? 3,2,1'
springs = 5*(line.strip().split(' ')[0]+'?')
springs = springs[:-1]
solution = 5*[int(x) for x in line.strip().split(' ')[1].split(',')]


min_runs = [x for x in springs.split('.') if x != '']
chunk_options = []

### Day 13

In [207]:
f = open('../data/day_13_input.txt').read()

patterns = [x.split('\n') for x in f.split('\n\n')]

cols_above = 0
rows_left = 0

idx = 0
for pattern in patterns:
    for fold in range(1, len(pattern)):
        top_half = pattern[:fold]
        bottom_half = pattern[fold:]

        if len(top_half) > len(bottom_half):
            top_half = top_half[-len(bottom_half):]
        else:
            bottom_half = bottom_half[:len(top_half)]

        bottom_half.reverse()

        if top_half == bottom_half:
            cols_above = cols_above + fold

    for fold in range(1, len(pattern[0])):
        left_half = [x[:fold] for x in pattern]
        right_half = [x[fold:] for x in pattern]

        if len(left_half[0]) > len(right_half[0]):
            left_half = [x[-len(right_half[0]):] for x in left_half]
        else:
            right_half = [x[:len(left_half[0])] for x in right_half]

        right_half = [x[::-1] for x in right_half]

        if left_half == right_half:
            rows_left = rows_left + fold

    idx = idx + 1

100*cols_above + rows_left

35360

In [208]:
f = open('../data/day_13_input.txt').read()

patterns = [x.split('\n') for x in f.split('\n\n')]

# counting up the new mirrors
cols_above = 0
rows_left = 0

# correcting the smudges
for i in range(len(patterns)):

    new_pattern = np.array([list(x.replace('#', '1').replace('.', '0')) for x in patterns[i]]).astype(int)

    corrected = 0
    for fold in range(1, new_pattern.shape[0]): 
        top_half = new_pattern[:fold]
        bottom_half = new_pattern[fold:]

        if top_half.shape[0] > bottom_half.shape[0]:
            top_half = top_half[-bottom_half.shape[0]:]
        else:
            bottom_half = bottom_half[:top_half.shape[0]]

        bottom_half = bottom_half[::-1]

        diffs = ((top_half - bottom_half)**2)

        if diffs.sum() == 1:
            smudge = np.argwhere(diffs != 0)[0]
            spot_to_change = new_pattern[fold - diffs.shape[0] + smudge[0], smudge[1]]
            if spot_to_change == 0:
                new_pattern[fold - diffs.shape[0] + smudge[0], smudge[1]] = 1
            else:
                new_pattern[fold - diffs.shape[0] + smudge[0], smudge[1]] = 0
            corrected = 1
            cols_above = cols_above + fold
            break

    if corrected == 0:
        for fold in range(1, new_pattern.shape[1]):
            left_half = new_pattern[:, :fold]
            right_half = new_pattern[:, fold:]

            if left_half.shape[1] > right_half.shape[1]:
                left_half = left_half[:, -right_half.shape[1]:]
            else:
                right_half = right_half[:, :left_half.shape[1]]

            right_half = right_half[:, ::-1]

            diffs = ((left_half - right_half)**2)

            if diffs.sum() == 1:
                smudge = np.argwhere(diffs != 0)[0]
                spot_to_change = new_pattern[smudge[0], fold - diffs.shape[1] + smudge[1]]
                if spot_to_change == 0:
                    new_pattern[smudge[0], fold - diffs.shape[1] + smudge[1]] = 1
                else:
                    new_pattern[smudge[0], fold - diffs.shape[1] + smudge[1]] = 0
                rows_left = rows_left + fold
                break

100*cols_above + rows_left


36755

### Day 14

In [65]:
f = open('../data/day_14_input.txt').read()

platform = np.array([list(x) for x in f.replace('O', '1').replace('.', '0').replace('#', '9').split('\n')]).astype(int)

In [66]:
def tilt(platform):

    new_platform = platform.copy()

    for col in range(platform.shape[1]):
        slice = platform[:, col]
        new_slice = platform[:, col].copy()

        rounds = np.where(platform[:, col] == 1)[0]
        cubes = np.where(platform[:, col] == 9)[0]
        cubes = [-1] + [x for x in cubes] + [platform.shape[0] + 1]

        for idx in range(len(cubes)-1):
            new_slice[cubes[idx]+1:cubes[idx+1]] = -np.sort(-new_slice[cubes[idx]+1:cubes[idx+1]])

        new_platform[:, col] = new_slice

    return new_platform

def cycle(platform):
    for i in range(4):
        platform = tilt(platform)
        platform = np.rot90(platform, 3)
    return platform

def north_load(platform):
    return (platform*np.array(range(platform.shape[0], 0, -1)).reshape((platform.shape[0], 1)))[platform==1].sum()


In [67]:
north_load(tilt(platform))

113525

In [90]:
# Work in progress - to answer the problem I did this manually in Excel the first time. 

# Need to write a function that collapses my array to a string and back again
mem = {}
for i in range(1, 251):
    platform = cycle(platform)
    platform_str = platform.tobytes()
    if platform_str in mem:
        first_repeat = mem[platform_str]
        period = i - mem[platform_str]
        break
    else:
        mem[platform_str] = i

index = {}
for s, i in mem.items():
    index[i] = s

def cycle_value(cycle_count):
    return north_load(np.frombuffer(index[(first_repeat + (cycle_count - first_repeat) % period)], like=platform))

print(cycle_value(1000000000))

0.0


### Day 15

In [87]:
def aoc_hash(string):
    current = 0
    for a in string:
        current = current + ord(a)
        current = 17*current
        current = current % 256
    return current

In [88]:
f = open('../data/day_15_input.txt').read()

score = 0
for x in f.strip().split(','):
    score = score + aoc_hash(x)

In [89]:
score

506437

In [90]:
f = open('../data/day_15_input.txt').read()

instructions = f.strip().split(',')

boxes = {x:{} for x in range(256)}

for sample in instructions:
    if sample[-1] == '-':
        action = '-'
        label = sample[:-1]
        focal_power = None
        box = aoc_hash(label)
        boxes[box].pop(label, None)
    else:
        action = '='
        label, focal_power = sample.split('=')
        box = aoc_hash(label)
        boxes[box].update({label:focal_power})

power = 0
for box in boxes:
    for i in range(len(boxes[box])):
        power = power + (box+1)*(i+1)*(int(list(boxes[box].values())[i]))

In [91]:
power

288521